# Curso Transfer Learning

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

## Creando nuestro propio Clasificador de Imágenes 🌷🌳

En este lab aprenderás:

* [Pytorch](https://pytorch.org/)
* [Torchvision](https://pytorch.org/vision/stable/index.html)
* Descargar un dataset, prepararlo, entrenarlo, realizar finetuning y guardarlo.


### 1) Descarga del dataset 🤓

Utilizaremos un conjunto de imágenes de enfermedades de las hojas de plantas.
<br>Para más detalle acá se puede ver el dataset de Kaggle: [Plant disease Dataset](https://www.kaggle.com/datasets/rashikrahmanpritom/plant-disease-recognition-dataset).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s plant-disease-classification

In [ ]:
!kaggle datasets download -d rashikrahmanpritom/plant-disease-recognition-dataset

In [ ]:
!unzip '/content/plant-disease-recognition-dataset.zip'

***Creamos un folder "enfermedades_hojas" para guardar nuestras imágenes y mantener una estructura similar a la del ejemplo anterior.***

In [ ]:
!mkdir /content/enfermedades_hojas

***Cambiamos el nombre de la carpeta "validation" por "val", para seguir la misma estructura del código de ejemplo***

In [ ]:
!mv /content/Train/Train /content/enfermedades_hojas/train/
!mv /content/Test/Test /content/enfermedades_hojas/test/
!mv /content/Validation/Validation /content/enfermedades_hojas/val/

***Removemos las carpetas vacías porque ya no los necesitamos.***

In [ ]:
!rm -r /content/Test
!rm -r /content/Train
!rm -r /content/Validation

### 2) Preparación de la data 👌

#### 2.1) Instalamos las dependencias 🙌

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import ResNet50_Weights, VGG19_Weights, MobileNet_V2_Weights

import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy

#### 2.2) Tips de implementación 👀

In [ ]:
cudnn.benchmark = True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### 2.3) Data Augmentation 📷 📸

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#### 2.4) Dataset, Dataloader y más data 🔍

In [ ]:
!ls

In [ ]:
data_dir = 'enfermedades_hojas'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

In [ ]:
def imshow(inp, title=None):
    """Imshow para tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # Pausarlo un poco para que se actualicen los plots


# Obtener un batch de datos de entrenamiento
inputs, classes = next(iter(dataloaders['train']))

# Crear una grilla con las imágenes
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

***Evaluar las class_names***

In [ ]:
print(class_names)

### 3) Entrenamiento 💪

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Cada epoch tiene una fase de entrenamiento y validación.
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Fase entrenamiento
            else:
                model.eval()   # Fase validación

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre la data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history solo si es la fase de entrenamiento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize solo si es la fase de entrenamiento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estadísticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy el modelo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Cargar el best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

### 4) Finetuning de la convnet 😨

#### 4.1) Resnet50

In [ ]:
model_ft__resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model_ft__resnet.fc.in_features

In [ ]:
# Reemplazar la capa final
# Aquí el tamaño de cada muestra de salida se establece en len(class_names) = 3.
model_ft__resnet.fc = nn.Linear(num_ftrs, len(class_names))

model_ft__resnet = model_ft__resnet.to(device)

criterion = nn.CrossEntropyLoss()

# Observe que todos los parámetros están siendo optimizados
optimizer_ft = optim.SGD(model_ft__resnet.parameters(), lr=0.001, momentum=0.9)

# Decae LR por un factor de 0.1 cada 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

***Vamos a usar 5 epochs***

In [ ]:
model_ft__resnet = train_model(model_ft__resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
visualize_model(model_ft__resnet)

Guardar el modelo

In [ ]:
torch.save(model_ft__resnet, "model_resnet50.pth")

Nota: El archivo de este modelo entrenado pesa 90 MB.

#### 4.2) Resnet50 + Custom Layers

In [ ]:
model_ft__custom = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model_ft__custom.fc.in_features

Definir un nuevo clasificador con una capa Dropout adicional

In [ ]:
model_ft__custom.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),       # Añadir una capa lineal intermedia
    nn.ReLU(),                      # Activación no lineal
    nn.Dropout(0.5),                # Dropout con probabilidad de 0.5
    nn.Linear(512, len(class_names))  # Capa final adaptada al número de clases
)

model_ft__custom = model_ft__custom.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

# Observe que todos los parámetros están siendo optimizados
optimizer_ft = optim.SGD(model_ft__custom.parameters(), lr=0.001, momentum=0.9)

# Decae LR por un factor de 0.1 cada 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

***Vamos a usar 5 epochs***

In [ ]:
model_ft__custom = train_model(model_ft__custom, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
visualize_model(model_ft__custom)

Guardar el modelo

In [ ]:
torch.save(model_ft__custom, "model_custom.pth")

Nota: El archivo de este modelo entrenado pesa 94 MB.

#### 4.3) VGG19

In [ ]:
model_ft__vgg19 = models.vgg19(weights=VGG19_Weights.IMAGENET1K_V1)
num_ftrs = model_ft__vgg19.classifier[6].in_features

In [ ]:
# Reemplazar la capa final
# Aquí el tamaño de cada muestra de salida se establece en len(class_names) = 3.
model_ft__vgg19.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model_ft__vgg19 = model_ft__vgg19.to(device)

criterion = nn.CrossEntropyLoss()

# Observe que todos los parámetros están siendo optimizados
optimizer_ft = optim.SGD(model_ft__vgg19.parameters(), lr=0.001, momentum=0.9)

# Decae LR por un factor de 0.1 cada 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

***Vamos a usar 5 epochs***

In [ ]:
model_ft__vgg19 = train_model(model_ft__vgg19, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
visualize_model(model_ft__vgg19)

Guardar el modelo

In [ ]:
torch.save(model_ft__vgg19, "model_vgg19.pth")

Nota: El archivo de este modelo entrenado pesa 532 MB.

#### 4.4) Mobilenet v2

In [ ]:
model_ft__mobilenetv2 = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V2)
num_ftrs = model_ft__mobilenetv2.last_channel

In [ ]:
# Reemplazar la capa final
# Aquí el tamaño de cada muestra de salida se establece en len(class_names) = 3.
model_ft__mobilenetv2.classifier[1] = nn.Linear(num_ftrs, len(class_names))

model_ft__mobilenetv2 = model_ft__mobilenetv2.to(device)

criterion = nn.CrossEntropyLoss()

# Observe que todos los parámetros están siendo optimizados
optimizer_ft = optim.SGD(model_ft__mobilenetv2.parameters(), lr=0.001, momentum=0.9)

# Decae LR por un factor de 0.1 cada 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

***Vamos a usar 5 epochs***

In [ ]:
model_ft__mobilenetv2 = train_model(model_ft__mobilenetv2, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
visualize_model(model_ft__mobilenetv2)

Guardar el modelo

In [ ]:
torch.save(model_ft__mobilenetv2, "model_mobilenetv2.pth")

Nota: El archivo de este modelo entrenado pesa ~9 MB.

### 5) Hacer Predicciones en Producción 🤙

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

Cargar el modelo una vez (al inicio de la aplicación)

In [ ]:
# Determinar el dispositivo
mi_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo en el dispositivo
mi_modelo = torch.load("model_mobilenetv2.pth", map_location=mi_device, weights_only=False)
mi_modelo.eval()
mi_modelo.to(mi_device)

Función para predicción / inferencia

In [ ]:
def predict_image(image_path, model, class_names, device):
    """
    Predice la clase de una imagen usando un modelo entrenado.

    Args:
        image_path (str): Ruta de la imagen.
        model (torch.nn.Module): Modelo ya cargado y configurado para inferencia.
        class_names (list): Lista de nombres de clases.
        device (torch.device): Dispositivo donde está el modelo.

    Returns:
        str: Clase predicha.
    """
    # Preprocesamiento de la imagen
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Cargar y preprocesar la imagen
    image = Image.open(image_path).convert("RGB")
    image_tensor = preprocess(image).unsqueeze(0).to(device)  # Añadir dimensión batch y mover al dispositivo

    # Realizar predicción
    with torch.no_grad():
        outputs = model(image_tensor)
        _, preds = torch.max(outputs, 1)

    # Retornar la clase predicha
    return class_names[preds[0].item()]

Ejemplos de uso

In [ ]:
class_names = ['Healthy', 'Powdery', 'Rust']

In [ ]:
path_image = "/content/enfermedades_hojas/val/Rust/8030a3a79fca6abb.jpg"

predicted_class = predict_image(path_image, mi_modelo, class_names, mi_device)
print(f"La clase predicha es: {predicted_class}")

In [ ]:
path_image = "/content/enfermedades_hojas/val/Healthy/9c3f1c10ba54ed56.jpg"

predicted_class = predict_image(path_image, mi_modelo, class_names, mi_device)
print(f"La clase predicha es: {predicted_class}")

In [ ]:
path_image = "/content/enfermedades_hojas/val/Powdery/8f6737815b2cd234.jpg"

predicted_class = predict_image(path_image, mi_modelo, class_names, mi_device)
print(f"La clase predicha es: {predicted_class}")

### 6) Conclusiones

- Aprender sobre los distintos objetos y métodos que nos ofrece Pytorch / Torchvision.

- Realizar el proceso completo de clasificación de imágenes con Pytorch.

- Aprender tips sobre implementación con el uso de la GPU.

<br>
<br>
<br>

---

<br>
<br>


<img src="https://static.platzi.com/media/avatars/platziteam_8cfe6fc7-1246-4c9a-9f5d-d10d467443ee.png" width="100px">

[Platzi](https://platzi.com/) 🚀

